In [107]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# import custom dependencies
from ADNI_utilities import define_terms, describe_meta_data, paths_with_ext, append_meta_cols

In [108]:
adni_dict_df = pd.read_csv("../data/Study Info/DATADIC.csv")
merge_dict_df = pd.read_csv("../data/Study Info/ADNIMERGE_DICT.csv")
lab_dict = pd.read_csv("../data/Biomarker Data/LABDATA_DICT.csv")

In [109]:
# get the feature importance
paths = paths_with_ext("../data/Models/Feature_Importance/")
fnames = []
top_features = []
importance = np.tile(np.nan, (3,176))
i=0
for path in paths:
    
    if "modeled_upto_50" in path and "Log" not in path:
        df=pd.read_csv(path)
        df = df.drop(columns="Unnamed: 0",axis=1)
        mean_importance = df.mean().values
        mean_importance = mean_importance/mean_importance.sum()
        importance[i,:] = mean_importance
        sorted_feats = df.columns[np.argsort(mean_importance)[::-1]]
        top_features.append(sorted_feats[:100])
        fnames.append(os.path.basename(path).split(".")[0])
        i+=1
        
importance_df = pd.DataFrame(data=importance, columns=df.columns, index=fnames)

In [110]:
# find common features
common_features = top_features[0]
for feats in top_features:
    common_features = list(set(common_features) & set(feats))

In [111]:
# find common features
common_features = top_features[0]
for feats in top_features:
    common_features = list(set(common_features) & set(feats))

In [112]:
imp = importance_df.loc["RandomForestClassifier_modeled_upto_50",common_features]
sorted_idx = np.argsort(imp.values)[::-1]
sorted_imp = imp.values[sorted_idx]
sorted_feats = [common_features[i] for i in sorted_idx]

In [113]:
df = pd.DataFrame(columns=sorted_feats, index=[0])
terms = define_terms(df, adni_dict_df)
features = df.columns
terms["FLDNAME"] = df.columns

# add in definitions from adnimerge dict
merge_terms = define_terms(df, merge_dict_df)
is_merge = ~merge_terms.FLDNAME.isna()
terms.loc[is_merge,"TEXT"] = merge_terms.loc[is_merge,"TEXT"]
terms.loc[is_merge,"CRFNAME"] = merge_terms.loc[is_merge,"CRFNAME"]

# reverse column order
terms['FEATURE_IMPORTANCE'] = sorted_imp
cols = ['FLDNAME', 'FEATURE_IMPORTANCE', 'TEXT', 'CRFNAME']
terms = terms[cols]


In [114]:
terms

,FLDNAME,FEATURE_IMPORTANCE,TEXT,CRFNAME
0,CDRSB,0.075535,CDR-SB,Key variables merged into one data table
1,mPACCdigit,0.059558,ADNI modified Preclinical Alzheimer's Cognitiv...,Key variables merged into one data table
2,LDELTOTAL,0.052368,Logical Memory - Delayed Recall,Key variables merged into one data table
3,mPACCtrailsB,0.049832,ADNI modified Preclinical Alzheimer's Cognitiv...,Key variables merged into one data table
4,FAQ,0.040017,FAQ,Key variables merged into one data table
5,MOCA,0.024542,MOCA,Key variables merged into one data table
6,EcogSPTotal,0.024169,SP ECog - Total,Key variables merged into one data table
7,ADASQ4,0.023870,ADAS Delayed Word Recall,Key variables merged into one data table
8,ADAS13,0.022962,ADAS 13,Key variables merged into one data table
9,FDG,0.022923,"Average FDG-PET of angular, temporal, and post...",Key variables merged into one data table


In [115]:
terms.to_csv("../data/Results/top_feature_definitions.csv")